In [1]:
import os
import pathlib

import pandas as pd
import math
from numpy import ndarray, unique, nan

from helpers.columns import COL_HEAD
from helpers.predict_data import predict, is_significant

In [2]:
def get_final_year():
    allowed_years = [COL_HEAD.YEAR_2020, COL_HEAD.YEAR_2021]
    msg = f'Please provide final year to analize: [allowed are: {allowed_years}] '

    get_year = input(msg)
    while get_year not in allowed_years:
        get_year = input(msg)
    
    print(f'Will analyze data for {get_year}')
    return get_year

In [3]:
def concat_column_vals(df: pd.DataFrame, main_col, additional_col, brackets):
    return df[main_col].map(str) + brackets[0] + df[additional_col].map(str) + brackets[1]

In [4]:
#  Get Cleaned Mortality from eurostat
cleaned_mortality_file = pathlib.Path(f'{os.getcwd()}/../../../cleaned_data/Municipalities - Mortality.csv')
df = pd.read_csv(cleaned_mortality_file)

In [5]:
compare_years = [COL_HEAD.YEAR_2015, COL_HEAD.YEAR_2016, COL_HEAD.YEAR_2017, COL_HEAD.YEAR_2018, COL_HEAD.YEAR_2019]

df[COL_HEAD.MEAN_OR_EXPECTED_MORTALITY] = df.loc[:,compare_years].mean(axis=1)

In [6]:
df[COL_HEAD.STANDARD_DEVIATION] = df.loc[:, compare_years].std(axis=1, ddof=0).round(1)

In [7]:
df[COL_HEAD.Z_SCORE] = 1.96

In [8]:
df[COL_HEAD.CONFIDENCE_INTERVAL] = df.apply(lambda x: 
                                            x[COL_HEAD.Z_SCORE] * (x[COL_HEAD.STANDARD_DEVIATION] / math.sqrt(5)),
                                            axis=1).round(1)

In [9]:
df[COL_HEAD.LB_MEAN_MORTALITY] = df[COL_HEAD.MEAN_OR_EXPECTED_MORTALITY] - df[COL_HEAD.CONFIDENCE_INTERVAL]
df[COL_HEAD.UB_MEAN_MORTALITY] = df[COL_HEAD.MEAN_OR_EXPECTED_MORTALITY] + df[COL_HEAD.CONFIDENCE_INTERVAL]

In [10]:
analyze_year = get_final_year()

df[COL_HEAD.EXCESS_MORTALITY_BASE] = df.apply(lambda x: 
                                                x[analyze_year] - x[COL_HEAD.MEAN_OR_EXPECTED_MORTALITY],
                                                axis=1).round(1)

Will analyze data for 2021


In [11]:
df[COL_HEAD.P_SCORE] = df.apply(lambda x:
                                (
                                        (x[analyze_year] - x[COL_HEAD.MEAN_OR_EXPECTED_MORTALITY])
                                        /
                                        x[COL_HEAD.MEAN_OR_EXPECTED_MORTALITY]
                                ) * 100
                                if x[COL_HEAD.MEAN_OR_EXPECTED_MORTALITY] != 0
                                else 0,
                                axis=1).round(1)

df[COL_HEAD.P_SCORE_FLUCTUATION] = df.apply(lambda x:
                                            x[COL_HEAD.P_SCORE]
                                            -
                                            (
                                                    (
                                                            (x[analyze_year] - x[COL_HEAD.UB_MEAN_MORTALITY])
                                                            / x[COL_HEAD.UB_MEAN_MORTALITY]
                                                    )
                                                    * 100
                                            )
                                            if x[COL_HEAD.UB_MEAN_MORTALITY] != 0
                                            else np.nan,
                                            axis=1).round(1)


In [12]:
df[COL_HEAD.MEAN_MORTALITY_DECORATED] = concat_column_vals(df,
                                                            COL_HEAD.MEAN_OR_EXPECTED_MORTALITY,
                                                            COL_HEAD.CONFIDENCE_INTERVAL,
                                                            [' (±', ')'])

df[COL_HEAD.EXCESS_MORTALITY_DECORATED] = concat_column_vals(df,
                                                            COL_HEAD.EXCESS_MORTALITY_BASE,
                                                            COL_HEAD.CONFIDENCE_INTERVAL,
                                                            [' (±', ')'])

df[COL_HEAD.P_SCORE_DECORATED] = concat_column_vals(df,
                                                    COL_HEAD.P_SCORE,
                                                    COL_HEAD.P_SCORE_FLUCTUATION,
                                                    ['% (±', '%)'])

In [13]:
file_name = f'Predicted_excess_mortality_by_Municipality:{analyze_year}.csv'
path = pathlib.Path(f'{os.getcwd()}/../../../output_data/Excess_mortality/{file_name}')
df.to_csv(path, index=False)